Linear Regression Model for Tourism Data 

In [11]:
import os  
import sys

os.getcwd()


'/home/clif_lastrophysicien/ELO2_LAPERLE_HT/4_data_analysis'

Step 1: Understanding the Data

In [15]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings("ignore")

# Set display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)


In [14]:
# Set style for better visualizations
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")


2: Loading and Exploring the Data

In [16]:
# Load the dataset
df = pd.read_csv("UN_tourism_caribbean_countries_cleaned.csv")

# Display basic information
print("=== DATASET INFORMATION ===")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nMissing Values:\n{df.isnull().sum()}")
print(f"\nFirst 5 rows:")
print(df.head())

print(f"\n=== BASIC STATISTICS ===")
print(df.describe())

print(f"\n=== UNIQUE VALUES ===")
print(f"Visitor types: {df['type_of_visitors'].unique()}")
print(f"Number of countries: {df['country_receiving'].nunique()}")
print(f"Years range: {df['year'].min()} to {df['year'].max()}")


=== DATASET INFORMATION ===
Shape: (3025, 6)

Columns: ['type_of_visitors', 'country_receiving', 'where_tourist_from', 'year', 'number_of_tourist', 'unit']

Data Types:
type_of_visitors       object
country_receiving      object
where_tourist_from     object
year                    int64
number_of_tourist     float64
unit                   object
dtype: object

Missing Values:
type_of_visitors      0
country_receiving     0
where_tourist_from    0
year                  0
number_of_tourist     0
unit                  0
dtype: int64

First 5 rows:
  type_of_visitors    country_receiving where_tourist_from  year  \
0    excursionists  Antigua and Barbuda              World  1995   
1    excursionists  Antigua and Barbuda              World  1996   
2    excursionists  Antigua and Barbuda              World  1997   
3    excursionists  Antigua and Barbuda              World  1998   
4    excursionists  Antigua and Barbuda              World  1999   

   number_of_tourist            unit  


In [20]:
data.info()

data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025 entries, 0 to 3024
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   type_of_visitors    3025 non-null   object 
 1   country_receiving   3025 non-null   object 
 2   where_tourist_from  3025 non-null   object 
 3   year                3025 non-null   int64  
 4   number_of_tourist   3025 non-null   float64
 5   unit                3025 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 141.9+ KB


,type_of_visitors,country_receiving,where_tourist_from,year,number_of_tourist,unit
0,excursionists,Antigua and Barbuda,World,1995,227.0,thousand trips
1,excursionists,Antigua and Barbuda,World,1996,270.0,thousand trips
2,excursionists,Antigua and Barbuda,World,1997,286.0,thousand trips
3,excursionists,Antigua and Barbuda,World,1998,336.0,thousand trips
4,excursionists,Antigua and Barbuda,World,1999,328.0,thousand trips


1. Understanding the Data Structure

  The dataset contains:

  type_of_visitors: Type of visitors (tourists, excursionists, visitors_total)

  country_receiving: Destination country

  where_tourist_from: Origin (all are "World")

  year: Year of data

  number_of_tourist: Number of visitors (in thousand trips)

  unit: Measurement unit (thousand trips)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset (assuming it's in a CSV file)
df = pd.read_csv("UN_tourism_caribbean_countries_cleaned.csv")

# Let's focus on predicting total visitors for a specific country
# We'll use "Dominican Republic" as an example since it has comprehensive data

# Filter for total visitors data for Dominican Republic
dr_data = df[
    (df["country_receiving"] == "Dominican Republic")
    & (df["type_of_visitors"] == "visitors_total")
].copy()

# Sort by year
dr_data = dr_data.sort_values("year")

# Prepare features (X) and target (y)
# We'll use year as the feature to predict number of tourists
X = dr_data[["year"]].values
y = dr_data["number_of_tourist"].values

# Check if we have enough data
print(f"Number of data points: {len(dr_data)}")
print(f"Years range: {dr_data['year'].min()} to {dr_data['year'].max()}")


Number of data points: 30
Years range: 1995 to 2024
